In [1]:
from binance.client import Client
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
import datetime
from matplotlib.finance import candlestick_ohlc, candlestick2_ohlc

/Users/Daniel/anaconda3/envs/carnegie/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
api_key = '8h0dbjBU8wkx6iqgHZT0dnVHdvuOHwENJBvteURTFh1QpbwNYn6DSHBaMx6ePJHa'
api_secret = 'DdwShMR2JK5sCrhRgygTlkc4sVCSZnLXHy84FJIifnuxTgokXOKDenZ7OLaFnzqs'

In [3]:
client = Client(api_key, api_secret)

In [102]:
candles = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_3MINUTE, "2 day ago UTC")

In [101]:
len(candles)

2880

In [10]:
candles = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1MINUTE, "10 hour ago UTC")
col_names = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'n_trades', 'taker_buy_base_volume', 'taker_buy_quote_volume']
lst = []
for candle in candles:
    lst.append(dict(zip(col_names, candle)))
data = pd.DataFrame(lst)
data['open_time'] = pd.to_datetime(data['open_time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
data['close_time'] = pd.to_datetime(data['close_time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
data = data.set_index('open_time')
data.index = pd.to_datetime(data.index, "%Y-%m-%d %h:%m%s")
data = data[['open', 'close', 'high', 'low', 'volume', 'taker_buy_base_volume']]
data['sell'] = data.volume.astype(np.float64) - data.taker_buy_base_volume.astype(np.float64)

data['low'] = data.low.astype(np.float64)
data['high'] = data.high.astype(np.float64)
data['close'] = data.close.astype(np.float64)
data['open'] = data.open.astype(np.float64)
data['volume'] = data.volume.astype(np.float64)
data['taker_buy_base_volume'] = data.taker_buy_base_volume.astype(np.float64)
data['sell'] = data.sell.astype(np.float64)

In [75]:
client = MongoClient("mongodb://mongo-7a3e43c9-1.bf9bfd39.cont.dockerapp.io:32794")
client.database_names()
full_set = pd.DataFrame(list(client.Binance['klines_BTCUSDT_1h'].find()))

In [138]:
def algo1(df, delta_n=5):

    lst = []
    
    open_series = df[['c_p']]
#     print(open_series)

    for idx in range(len(open_series['c_p']) - delta_n):
        selected = open_series['c_p'][idx:idx+(delta_n+1)].values
        sorted_val = np.sort(selected[1:])
        lst.append(np.mean(sorted_val[1:delta_n-1]) - selected[0])

    df['y'] = np.append(lst, [np.nan] * delta_n)
    sub_set = df[['o_t', 'c_p', 'o_p', 'y']]
    sub_set['c_p'] = sub_set['c_p'].astype(np.float)
    sub_set['o_p'] = sub_set['o_p'].astype(np.float)
    sub_set['temp'] = 1
    sub_set['temp'][sub_set.y < 0] = 0
    sub_set['signal'] = np.append([0], np.negative(np.diff(sub_set.temp)))
    return sub_set


In [139]:
def simul(sub_set):
    profit = np.sum(sub_set['o_p'] * sub_set['signal'])
    return profit

In [140]:
sub_set = algo1(full_set)

/Users/Daniel/anaconda3/envs/carnegie/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Daniel/anaconda3/envs/carnegie/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Daniel/anaconda3/envs/carnegie/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [141]:
simul(sub_set)

-22714.849999999984

In [ ]:
# def algo1():
#     full_set['c_p'] = full_set['c_p'].astype(np.float)
#     sub_set['o_p'] = sub_set['o_p'].astype(np.float64)
#     full_set_aug = generate_df(full_set)
#     sub_set = full_set_aug[['o_t', 'c_p', 'o_p', 'y']]
#     sub_set['signal'] = 1
#     sub_set['signal'][sub_set.y < 0] = 0
#     sub_set['diff'] = np.append([0], np.negative(np.diff(sub_set.signal)))
#     return sub_set
